# RedGrease Demo for RedisConf 2021

Quick demonstration of how to create and run Redis Gears functions, using RedGrease.

Demos:
- Simple Gears Query (link)
- Transactions Stream processing (link)
- Custom command (link)

# Preparations
Before running the demos, make sure that the prerequisites are met and that the preparation steps have successfully been executed. 
Some preparation steps, particularly the downloads, may take quite some time. 

## Prerequisites
- Python3.7
- Pip
- Docker

## Download Datasets
## Download Redis Gears Cluster Docker image
## Run Redis Gears Cluster contaioner
## Install Python requirements
## Load Data into cluster

# Demos
## 1. Simple Gears Query
## 2. Transactions Stream Processing 
## 3. Custom command

In [2]:
import sys
!{sys.executable} --version
!{sys.executable} -m pip --version
!docker --version

Python 3.7.10
pip 20.2.2 from /home/anders/projects/redgrease/.venv/lib/python3.7/site-packages/pip (python 3.7)
Docker version 20.10.5, build 55c4c88


## 1. Install Dependencies

In [3]:
import sys
!{sys.executable} -m pip install redgrease[client]

     |████████████████████████████████| 70 kB 5.0 MB/s 
     |████████████████████████████████| 41 kB 339 kB/s 
You should consider upgrading via the '/home/anders/projects/redgrease/.venv/bin/python -m pip install --upgrade pip' command.


In [4]:
# Run a minimal (3 shard) Redis Gears cluster
!docker run --name gears --rm -d -p 30001:30001 -p 30002:30002 -p 30003:30003 redislabs/rgcluster:latest 

32a741d6d76380730daf98ec76d06e06c1e300360eaa7692189b07165c0b8f10


In [5]:
import redgrease
import redgrease.utils

In [6]:
r = redgrease.Redis(host="localhost", port=30001)

In [7]:
r.set("Anders", 41)

ResponseError: MOVED 15027 172.17.0.2:30003

In [9]:
relevant_usr_fields = {
    "active": bool,
    "permissions": redgrease.utils.list_parser(str),
}


In [10]:
active_users = (
    redgrease.KeysOnlyReader("user:*")
    .map(lambda key: redgrease.cmd.hmget(key, *relevant_usr_fields.keys()))
    .map(
        lambda udata: redgrease.utils.to_dict(
            udata, keys=relevant_usr_fields.keys(), val_transform=relevant_usr_fields
        )
    )
    .filter(lambda usr: usr["active"])
)

In [11]:
active_user_count = active_users.count()

all_issued_permissions = active_users.flatmap(lambda usr: usr["permissions"]).distinct()

In [12]:
r = redgrease.client.RedisGears()

In [ ]:
count = r.gears.pyexecute(active_user_count)